Excel-To-Platform-Loan-Analytics Validation

In [21]:
import pandas as pd

borrowers = pd.read_csv("data/raw/borrowers.csv")
facilities = pd.read_csv("data/raw/facilities.csv")
loans = pd.read_csv("data/raw/loans.csv")
rates = pd.read_csv("data/raw/rates.csv")
cashflows = pd.read_csv("data/raw/cashflows.csv")

borrowers.head()

,borrower_id,borrower_name,industry,domicile_country,sponsor_name
0,B001,Atlas Capital Manufacturing,Industrials,US,NorthBridge Partners
1,B002,Hudson Infrastructure Holdings,Infrastructure,US,RiverRock Capital
2,B003,Meridian Healthcare Credit,Healthcare,US,StonePeak Advisors


This code simulates a platform ingestion step. I load normalized datasets from the raw data layer into pandas DataFrames, establishing a deterministic working dataset. The head() call simply verifies successful ingestion and structural integrity before applying validation or reconciliation logic.

In [9]:
facilities[~facilities.borrower_id.isin(borrowers.borrower_id)]

,facility_id,borrower_id,currency,commitment,start_date,maturity_date


This statement performs a referential integrity check between facilities and borrowers. It identifies facility records whose borrower_id values do not exist in the borrower master table, effectively detecting orphaned records that would violate platform relational rules.

In [11]:
loans[~loans.facility_id.isin(facilities.facility_id)]

,loan_id,facility_id,tranche,lien,status,spread_bps,base_rate_index


This line performs a referential integrity check between the loans and facilities datasets. It isolates loan records whose facility_id values do not exist in the facilities table, effectively detecting orphaned loans that would violate relational constraints within a production data platform.

In [13]:
cashflows[~cashflows.loan_id.isin(loans.loan_id)]

,cashflow_id,loan_id,cashflow_date,interest_amount,principal_amount,fee_amount


In [16]:
cashflows[["interest_amount","principal_amount","fee_amount"]].sum()


interest_amount     433000
principal_amount         0
fee_amount            5000
dtype: int64

In [24]:
validation_summary = {
"Borrower Duplicates": borrowers.borrower_id.duplicated().sum(),
"Orphan Facilities": len(facilities[~facilities.borrower_id.isin(borrowers.borrower_id)]),
"Orphan Loans": len(loans[~loans.facility_id.isin(facilities.facility_id)]),
"Orphan Cashflows": len(cashflows[~cashflows.loan_id.isin(loans.loan_id)])
}

validation_summary

{'Borrower Duplicates': 0,
 'Orphan Facilities': 0,
 'Orphan Loans': 0,
 'Orphan Cashflows': 0}

This statement performs a deterministic reconciliation check by aggregating key transactional measures across the cashflows dataset. Rather than relying on spreadsheet formulas, the platform computes control totals programmatically to ensure financial consistency, reproducibility, and auditability of interest, principal, and fee values.

Overview: To demonstrate platform-oriented thinking, I built a simplified analytics environment using normalized private credit datasets. The notebook models deterministic ingestion from a raw data layer, followed by structured validation routines including primary key integrity checks, referential consistency tests across hierarchical entities, and reconciliation controls on transactional cashflows.

Rather than relying on spreadsheet formulas, the workflow uses reproducible programmatic logic to detect structural breaks, orphaned records, and aggregation inconsistencies — mirroring how production data platforms enforce reliability and data integrity.

What Each Validation Category Represents:

Data Ingestion
"I explicitly load structured datasets representing entity and transactional tables."
Entity Integrity
"I validate identity uniqueness to prevent duplication and aggregation distortion."
Referential Integrity
"I enforce relational correctness across borrowers, facilities, and loans."
Reconciliation Controls
"I compute deterministic financial totals independent of spreadsheet logic."
Exception Detection
"I isolate structural failures rather than assuming clean data."